In [ ]:
! python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util

nlp = spacy.load("ru_core_news_lg")
model = SentenceTransformer('sberbank-ai/sbert_large_nlu_ru')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

# Extraction of skills

In [ ]:
import spacy
import re
import dateparser
from datetime import datetime
from sentence_transformers import SentenceTransformer, util


def extract_fragments(text):
    doc = nlp(text)
    fragments = []
    for sent in doc.sents:
        parts = re.split(r",| и | но | также |;|\n", sent.text)
        for part in parts:
            cleaned = part.strip()
            if len(cleaned.split()) >= 1:
                fragments.append(cleaned)
    return fragments


def get_skills(fragments, reference_vectors, threshold=0.7):
    results = {}
    for frag in fragments:
        frag_vec = model.encode(frag, convert_to_tensor=True)
        for skill in reference_vectors:
            cos_score = util.pytorch_cos_sim(frag_vec, tensor(reference_vectors[skill]))[0][0]
            if cos_score > threshold:
                results.add((frag, skill, similarity))
    return results


def extract_info(text):
    #Structure of returning data
    info = {
        "PER": '',
        "ORG": [],
        "EMAIL": [],
        "PHONE": [],
        "HARD_SKILLS": [],
        "SOFT_SKILLS": []
    }

    doc = nlp(text)

    # ФИО
    for ent in doc.ents:
        if ent.label_ == "PER":
            info["PER"] = ent.text
            break

    # ORGANIZATIONS + EXPERIENCE
    for ent in doc.ents:
        if ent.label_ == "ORG":
            info["ORG"].append[ent.text]

    # EMAIL
    info["EMAIL"] = re.findall(r'[\w\.-]+@[\w\.-]+', text)

    # PHONE
    info["PHONE"] = re.findall(r'\+?\d[\d\-\s\(\)]{7,}\d', text)

    # SKILLS
    fragments = extract_fragments(text)

    info["HARD_SKILLS"] = get_skills(fragments, hard_skills)
    info["SOFT_SKILLS"] = get_skills(fragments, soft_skills)

    return info


Участвовал в командных проектах работа в команде 0.3811096
Участвовал в командных проектах лидерство 0.19410764
Участвовал в командных проектах стрессоустойчивость -0.08129501
Участвовал в командных проектах навыки общения 0.12859333
умею управлять людьми работа в команде 0.2796116
умею управлять людьми лидерство 0.12476285
умею управлять людьми стрессоустойчивость -0.03322081
умею управлять людьми навыки общения 0.4146215
работать под давлением. работа в команде 0.4286022
работать под давлением. лидерство -0.0026698583
работать под давлением. стрессоустойчивость -0.16773133
работать под давлением. навыки общения 0.11412513


In [ ]:
import fitz

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_file = "your_file.pdf"
text = extract_text_from_pdf(pdf_file)
info = extract_info(text)

# Generation of skills sets

In [ ]:
hard_skills = {}

In [ ]:
from os import wait
import requests

def get_vacancies(text_query="Python разработчик", per_page=10, area=113):
    """
    Get some skills from HeadHunter by using API
    area=113 — Russia
    """
    url = "https://api.hh.ru/vacancies"
    params = {
        "text": text_query,
        "area": area,
        "per_page": per_page,
        "page": 0
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    vacancies = []
    for item in data["items"]:
        vacancy_detail = requests.get(item["url"]).json()
        skills = vacancy_detail.get("key_skills")
        if skills:
            for new_skill in map(lambda x: x["name"], skills) - hard_skills.keys():
                new_skill_vec = model.encode(new_skill, convert_to_tensor=True)
                unic = True
                for skill in soft_skills:
                    cos_score = util.pytorch_cos_sim(new_skill_vec, tensor(skills[skill]))[0][0]
                    if cos_score > 0.7:
                        print(new_skill, "is", skill)
                        unic = False
                        break
                if unic:
                  hard_skills[new_skill] = new_skill_vec.tolist()
                  print(type(new_skill_vec))

for _ in range(10):
  vacancies = get_vacancies("", per_page=50)

with open("hard_skills.json", "w", encoding="utf-8") as f:
    json.dump(hard_skills, f, ensure_ascii=False, indent=4)

NameError: name 'emb_text' is not defined

In [ ]:
soft_skills = """Активное слушание

Грамотная речь

Лидерство

Умение ясно выражать мысли

Вербальное общение

Жестовый язык

Иностранный язык

Написание деловой переписки

Умение задавать вопросы

Аргументация и убеждение

Обратная связь

Диалог и дискуссия

Умение адаптировать стиль общения под аудиторию

Самоосознание эмоций

Контроль над эмоциями

Эмпатия

Чувствительность к настроению других

Управление стрессом

Способность к самоанализу

Поддержка мотивации в себе и других

Разрешение эмоциональных конфликтов

Социальная адаптация

Чуткость к невербальным сигналам

Делегирование задач

Поддержка коллег

Умение работать в мультикультурной среде

Взаимная ответственность

Совместное принятие решений

Конструктивное взаимодействие

Гибкость в ролях

Обсуждение и компромиссы

Разрешение групповых конфликтов

Вдохновение других

Управление конфликтами

Наставничество

Принятие ответственности

Ведение изменений

Умение ставить цели

Принятие решений

Умение выслушать

Уверенность в себе

Приоритизация задач

Делегирование

Избежание прокрастинации

Умение говорить «нет»

Использование тайм-менеджмент инструментов

Установка дедлайнов

Контроль над отвлекающими факторами

Оценка времени выполнения задач

Баланс работы и отдыха

Выявление логических ошибок

Сравнение альтернатив

Обоснование выводов

Скептический подход

Проверка фактов

Системное мышление

Прогнозирование последствий

Оценка рисков

Вопросы «почему» и «как»

Открытость к новым идеям

Управление изменениями

Принятие обратной связи

Обучаемость

Многозадачность

Переключение между задачами

Умение работать в неопределенности

Креативный подход к проблемам

Позитивное отношение к изменениям

Генерация идей

Оценка вариантов

Выбор оптимального решения

Анализ причин и следствий

Тестирование решений

Учёт ресурсов

Управление рисками

Работа с ограничениями

Обратная связь и корректировка

Мозговой штурм

Визуализация концепций

Аналогии и метафоры

Комбинирование разных идей

Игра с гипотезами

Эксперименты

Свободное ассоциирование

Решение задач нестандартным способом

Критика идей без оценки

Умение сохранять спокойствие

Позитивное мышление

Распознавание признаков стресса

Тайм-ауты и перерывы

Разделение работы и личной жизни

Поддержка коллег в стрессовых ситуациях

Поиск помощи при необходимости

Медитация и релаксация

Принятие неопределенности

Эмоциональный отклик

Использование историй (сторителлинг)

Установление доверия

Акцент на выгодах

Подстройка под собеседника

Выявление потребностей

Умение слушать

Вопросы для мотивации

Работа с возражениями

Активное слушание сторон

Выражение своего мнения без агрессии

Поиск компромиссов

Посредничество

Эмоциональный контроль

Умение прощать

Разделение людей и проблемы

Профилактика конфликтов

Разработка решений, устраивающих всех

Отслеживание прогресса

Поощрение себя за достижения

Позитивное внутреннее общение

Умение работать без внешнего контроля

Настойчивость

Самокритика без самоосуждения

Управление энергией

Поиск смысла в задачах

Формирование привычек

Визуальное оформление слайдов

Выразительная речь

Контроль темпа и интонации

Работа с аудиторией

Использование вопросов

Управление временем презентации

Обработка вопросов и возражений

Уверенный язык тела

Использование технических средств

Вежливость и уважение

Быстрое реагирование на запросы

Персонализация общения

Умение слушать

Работа с возражениями

Предоставление решений, выгодных клиенту

Обратная связь

Построение долгосрочных отношений

Контроль качества обслуживания"""

In [ ]:
import json

# Предположим, soft_skills — это текст со скиллами через '\n\n'
soft_skills = soft_skills.split('\n\n')
soft_skills_json = {}

for skill in soft_skills:
    embedding = model.encode(skill, convert_to_tensor=True)
    soft_skills_json[skill] = embedding.tolist()  # Преобразуем тензор в список

# Сохраняем словарь: {skill: [vector]}
with open("soft_skills.json", "w", encoding="utf-8") as f:
    json.dump(soft_skills_json, f, ensure_ascii=False, indent=4)
